In [1]:
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

# Create a variable used as cutoff date for the data
cutoff_date_min = '1970-12-31'
cutoff_date_max = '2023-12-31'

# Portfolio Creation Jupyter Notebook
This notebook serves as an appendix for my master's thesis. It displays the methodology used to create industry portfolios. If readers would like to replicate the study, the data can be obtained using the approach described below:
#### Stock Returns
The stock returns data are obtained from WRDS. I have downloaded the complete dataset from: *Home / Get Data / CRSP / Annual Update / Stock / Security Files / Monthly Stock File*
The link is: https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-security-files/monthly-stock-file/
Certain stocks have subsequently been removed within this Jupyter Notebook.
#### Firm Fundamentals
The firm fundamentals data are obtained from WRDS. I have downloaded the complete dataset from: *Home / Get Data / CRSP / Annual Update / CRSP/Compustat Merged / Fundamentals Quarterly*
The link is: https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/crspcompustat-merged/fundamentals-quarterly/
#### SIC code index
TThe SIC code index is the industry segmentation used by Fama & French. The data can be obtained from the Kenneth R. French Data Library: *Data Library / U.S. Research Returns / Industry Portfolios / 12 Industry Portfolios / Details*
The link is: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_12_ind_port.html
### Cutoff Dates
I have set cutoff dates to ensure data availability, as most balance sheet information (total liabilities and total assets) was unavailable prior to 1971.

In [2]:
# Define paths to data
path_stock_returns ='C:\\Users\\valde\\OneDrive\\01 Master Thesis\\01 Data\\Data 240426\\Stock returns 240426.csv'
path_sic_12 = 'C:\\Users\\valde\\OneDrive\\01 Master Thesis\\01 Data\\Sic indicies\\Sic_12_df.csv'
path_firm_funda = 'C:\\Users\\valde\\OneDrive\\01 Master Thesis\\01 Data\\Data 240426\\Firm Funda 240426.csv'

# Load data
df_stock_returns = pd.read_csv(path_stock_returns)
df_sic_12 = pd.read_csv(path_sic_12)
df_firm_funda = pd.read_csv(path_firm_funda)

# Apply the cut off dates
df_stock_returns = df_stock_returns[(df_stock_returns['date'] >= cutoff_date_min) & (df_stock_returns['date'] <= cutoff_date_max)]
df_firm_funda = df_firm_funda[(df_firm_funda['datadate'] >= cutoff_date_min) & (df_firm_funda['datadate'] <= cutoff_date_max)]

## Data Merge
### Firm Fundamentals & SIC index
I start by preparing the firm fundamentals data set. I ensure that the SIC codes data type for both string for a succesfull data merge. As the SIC codes for *Other* industry portfolio is not defined i have simply assigned all SIC codes not included in the index to *Other* industry. I have ensured that there are no missing values before peforming this. 

In [3]:
# Delte columns that will not been used
df_firm_funda = df_firm_funda[['datadate','LPERMNO', 'LPERMCO', 'datafqtr', 'atq', 'ltq','sic']] 

# change the date type of 'sic' to string
df_firm_funda['sic'] = df_firm_funda['sic'].astype(str)
df_sic_12['sic'] = df_sic_12['sic'].astype(str)

# Merge the df
df=pd.merge(df_firm_funda,df_sic_12, on=['sic'], how='left')

# Reneme the column 'sector_12' to 'sector'
df = df.rename(columns={'sector_12': 'sector'})

# Assign missing values to the column 'sector' as 'Other'
df['sector'] = df['sector'].fillna('Other')

# Display the data
display(df.head())

,datadate,LPERMNO,LPERMCO,datafqtr,atq,ltq,sic,sector
0,1970-12-31,25881,23369,1970Q4,NaN,NaN,3089,Manuf
1,1971-03-31,25881,23369,1971Q1,NaN,NaN,3089,Manuf
2,1971-06-30,25881,23369,1971Q2,NaN,NaN,3089,Manuf
3,1971-09-30,25881,23369,1971Q3,NaN,NaN,3089,Manuf
4,1971-12-31,25881,23369,1971Q4,29.33,20.949,3089,Manuf


### Preparing the Stock Returns data
I prepare the stock returns data by performing simple data treatment. I scale the returns with 100 to obtain the returns in percentage. I will delete all duplicates of PERMCO within the same date as these duplicates display different stock class of the same stock. I start by sorting by the number of shares outstanding in descending order to ensure I will keep the biggest stock class.

In [4]:
# Apply the exchange criteria
df_stock_returns = df_stock_returns[df_stock_returns['EXCHCD'].isin([1, 2, 3])] # Delte all observation where 'EXCHCD' is not 1, 2 or 3
df_stock_returns['EXCHCD'] = df_stock_returns['EXCHCD'].replace({1: 'NYSE', 2: 'AMEX', 3: 'NASDAQ'}) # Change the name of the values in 'EXCHCD'

# Delte the columns: 'PRIMEXCH', 'TRDSTAT', 'FACPR', 'RETX', 'vwretx', 'ewretd', 'ewretx', 'sprtrn', 'FACSHR', 'SHRENDDT','SECSTAT'
df_stock_returns = df_stock_returns.drop(['PRIMEXCH', 'TRDSTAT', 'FACPR', 'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn','FACSHR','SHRENDDT','SECSTAT'], axis=1)

# Convert 'RET' to numeric, coercing errors to NaN (not a number) drop NaN
df_stock_returns['RET'] = pd.to_numeric(df_stock_returns['RET'], errors='coerce')
df_stock_returns.dropna(subset=['RET'], inplace=True)
df_stock_returns['RET'] = df_stock_returns['RET'] * 100

# Sort the data by 'PERMCO' and 'SHROUT' in descending order
df_stock_return = df_stock_returns.sort_values(['PERMCO', 'SHROUT'], ascending=[True, False])

# Delete duplicates of 'PERMCO' and 'date' and keep the first observation
df_stock_returns = df_stock_returns.drop_duplicates(subset=['PERMCO', 'date'], keep='first')

# Rename some of the columns
df_stock_returns = df_stock_returns.rename(columns={'date': 'Date', 'PERMCO':'LPERMCO', 'PERMNO':'LPERMNO'})

# Create a year & quarter variable to be merged on later
df_stock_returns['datafqtr'] = pd.to_datetime(df_stock_returns['Date']).dt.to_period('Q')

### Calculating the Market Capitalization
I calculate the market capitalization of each stock. First I adjust the prices and number of outstanding shares for stock split etc. I remove all observations were the closing price is less the 0.01 USD. If there is no closing price of a stock on a day CRSP denotes a minus sign before the price (Bid/Ask) I take the absoulte value as the price is not negative. I use the lagged market capitalization as these are input for the weights and these are calculated one period prior.

In [5]:
# Convert PRC to absolute value
df_stock_returns['PRC'] = df_stock_returns['PRC'].abs()

# Delete observations where 'PRC' is less than 0.01
df_stock_returns = df_stock_returns[df_stock_returns['PRC'] >= 0.01]

# Calculating the adjusted prices and shares outstanding
df_stock_returns['adj_prc'] = df_stock_returns['PRC'] / df_stock_returns['CFACPR']
df_stock_returns['adj_shrout'] = df_stock_returns['SHROUT'] * df_stock_returns['CFACSHR']

# Calculating the MktCap
df_stock_returns['MktCap'] = df_stock_returns['adj_prc'] * df_stock_returns['adj_shrout']

# Calculate lagged Market Capitalization
df_stock_returns['Date'] = pd.to_datetime(df_stock_returns['Date']) # Convert 'date' to datetime
df_stock_returns.sort_values('Date', inplace=True) # Ensure the data is correctly sorted
df_stock_returns['MktCap'] = df_stock_returns.groupby('LPERMCO')['MktCap'].shift(1) # Shift the data one period grouped by 'LPERMCO'

### Lag of data
I observe that there is approximately 4,000 companies more in the stock return data compared to the firm fundamentals. I am aware of this potential data bias, however, the missing companies do not have a biased toward certain sector nor substantially different returns. The missing companies are primarily listed on Nasdaq, this could indicate that the missing companies primarily consist of smaller companies. As I use value weighted return in my analysis this effect is assumed to be small.

In [6]:
# Count the number of unique 'LPERMCO' for both df_stock_returns and df_firm_funda
print('Number of unique LPERMCO in df_stock_returns:', df_stock_returns['LPERMCO'].nunique())
print('Number of unique LPERMCO in df_firm_funda:', df_firm_funda['LPERMCO'].nunique())

Number of unique LPERMCO in df_stock_returns: 30748
Number of unique LPERMCO in df_firm_funda: 26543


### Merging the data and fill NaNs
I observe that companies did not report firm fundamentals on a quarterly basis in the beging of the investigaion period. I have simply filled these missing value with existing values within the same year. I have simply used the calendar year and not the fiscal year, companies that reports EOY will therefore have a slighty lower asset value compared to companies that report BOY of MOY for instance (assuming the assets are increasing averagely). However as I don't use the nominal value of assets (or liabilities) but only the porportion this is assumed to have little effect on the overall results.
#### Debt ratio
I calculate the debt ratio as the total assets porportion to total liabilities
#### Deleting observations with missing values
I delete all missing values as these are fundamental for the rest of the analysis and it does not make sense to assign average/median values.
#### Deleting duplicates of LPERMCO
As there are also duplicates within the firm fundamentals i delete duplicates again, using the same approach

In [7]:
# Delete the column LPERMNO as it won't be used
df_stock_returns = df_stock_returns.drop('LPERMNO', axis=1)
df_firm_funda = df_firm_funda.drop('LPERMNO', axis=1)

# Merge the data
df['datafqtr'] = pd.to_datetime(df['datafqtr']).dt.to_period('Q') # Ensure the merging column has identical data format
df = pd.merge(df_stock_returns, df, on=['datafqtr', 'LPERMCO'], how='left')

# Extract year from 'Date'
df['Date'] = pd.to_datetime(df['Date']) # Ensure 'Date' is the correct data format
df['Year'] = df['Date'].dt.year

# Fill missing values in 'ltq' and 'atq' within the same year
df['ltq'] = df.groupby(['Year','LPERMCO'])['ltq'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
df['atq'] = df.groupby(['Year','LPERMCO'])['atq'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

# Calculate the debt ratio
df['DR'] = df['ltq'] / df['atq']
df['DR']= df['DR'].replace([np.inf, -np.inf], np.nan)

# Delete duplicates of 'PERMCO' and 'date' and keep the first observation
df['Date'] = pd.to_datetime(df['Date']) # Ensure 'Date' is the correct data format
df = df.sort_values(['LPERMCO', 'SHROUT'], ascending=[True, False])
df = df.drop_duplicates(subset=['LPERMCO', 'Date'], keep='first')

# Delete observations with missing values
df = df.dropna()

# Print the lenght of each df
print("Merged data set number of observations: ",len(df))

### Creating Debt-Ratio Portfolios
I am creating portfolios segmented on debt-ratio levels. I divide the data into three equally sized groups based on the debt-ratio levels. I subsequently calculate the weight of each stock within its respective debt-ratio segment.
### Creating Debt-Ratio Portfolios Within Sectors
I use the similar approach to create Debt-Ratio Portfolios within each sector.
### No missing values
There are no missing values in the dataset and the sums of the weights equal to 1 for all portfolios.

In [ ]:
# Assign 'high', 'mid', 'low' portfolio based on Debt-Ratio for each date
df['DR_port'] = df.groupby('datafqtr')['DR'].transform(lambda x: pd.qcut(x, 3, labels=['low', 'mid', 'high']))

# Assign 'high', 'mid', 'low' portfolio based on Debt-Ratio for each date grouped by sector
df['DR_sector_port'] = df.groupby(['datafqtr','sector'])['DR'].transform(lambda x: pd.qcut(x, 3, labels=['low', 'mid', 'high']))
df['DR_sector_port'] = df['sector'].astype(str) + '_' + df['DR_sector_port'].astype(str) # Change the name for better readability

# Calculate the weights
df['w_DR'] = df.groupby(['Date', 'DR_port'])['MktCap'].transform(lambda x: x / x.sum())
df['w_sector'] = df.groupby(['Date', 'sector'])['MktCap'].transform(lambda x: x / x.sum())
df['w_DR_sector'] = df.groupby(['Date', 'DR_sector_port'])['MktCap'].transform(lambda x: x / x.sum())
df['w_overall'] = df['MktCap'] / df.groupby('Date')['MktCap'].transform('sum')

# Print the number of missing values of the weights and ensure that the total weights of each group sums to 1
# Debt-Ratio portfolios
print('Number of missing weights in debt-ratio portfolios:', df['w_DR'].isnull().sum())
weight_sum = df.groupby(['Date', 'DR_port'])['w_DR'].sum().reset_index()
weight_sum.rename(columns={'w_DR': 'Total_Weight'}, inplace=True)
print('Number of Total_Weight that is not equal to 1(Debt-Ratio):', (weight_sum['Total_Weight'].round(10) != 1).sum())

# Sector portfolios
print('Number of missing weights in sector portfolios:', df['w_sector'].isnull().sum())
weight_sum = df.groupby(['Date', 'sector'])['w_sector'].sum().reset_index()
weight_sum.rename(columns={'w_sector': 'Total_Weight'}, inplace=True)
print('Number of Total_Weight that is not equal to 1 (sector):', (weight_sum['Total_Weight'].round(10) != 1).sum())

# Debt-ratio portfolios segmented by sector
print('Number of missing weights in debt-ratio portfolios segmented by sector:', df['w_DR_sector'].isnull().sum())
weight_sum = df.groupby(['Date', 'sector','DR_sector_port'])['w_DR_sector'].sum().reset_index()
weight_sum.rename(columns={'w_DR_sector': 'Total_Weight'}, inplace=True)
print('Number of Total_Weight that is not equal to 1 (DR segmented by sector):', (weight_sum['Total_Weight'].round(10) != 1).sum())

# Complete market
print('Number of missing weights in the complete market:', df['w_overall'].isnull().sum())
weight_sum = df.groupby('Date')['w_overall'].sum().reset_index()
weight_sum.rename(columns={'w_overall': 'Total_Weight'}, inplace=True)
print('Number of Total_Weight that is not equal to 1 (Complete market):', (weight_sum['Total_Weight'].round(10) != 1).sum())

Number of missing weights in debt-ratio portfolios: 0
Number of Total_Weight that is not equal to 1(Debt-Ratio): 0
Number of missing weights in sector portfolios: 0
Number of Total_Weight that is not equal to 1 (sector): 0
Number of missing weights in debt-ratio portfolios segmented by sector: 0
Number of Total_Weight that is not equal to 1 (DR segmented by sector): 0
Number of missing weights in the complete market: 0
Number of Total_Weight that is not equal to 1 (Complete market): 0


In [ ]:
# create copy data set were sector == 'Telcm'
df_telcm = df[df['sector'] == 'Telcm']

df_telcm = df_telcm.sort_values(['Date', 'DR_sector_port'], ascending=True)

display(df_telcm.head(20))

,Date,EXCHCD,SICCD,COMNAM,LPERMCO,PRC,RET,SHROUT,CFACPR,CFACSHR,...,sic,sector,Year,DR,DR_port,DR_sector_port,w_DR,w_sector,w_DR_sector,w_overall
3178,1971-01-29,NYSE,3661.0,GENERAL TEL & ELECTRS CORP,20776,31.0000,1.639300e-08,106359.0,3.000000,3.000000,...,4813,Telcm,1971,0.693016,high,Telcm_high,0.039771,0.290633,0.658572,0.007323
3989,1971-01-29,AMEX,4833,REEVES TELECOM CORP,24106,3.2500,3.000000e-07,2288.0,1.000000,1.000000,...,4832,Telcm,1971,0.983408,high,Telcm_high,0.000070,0.000512,0.001161,0.000013
2677,1971-01-29,NYSE,6711,UNITED UTILITIES INC,21833,23.2500,1.204820e-07,31523.0,5.931519,4.000000,...,4812,Telcm,1971,0.697892,high,Telcm_high,0.005408,0.039519,0.089551,0.000996
2918,1971-01-29,AMEX,4811,PHILIPPINE LONG DISTANCE TEL,21399,6.0000,9.090900e-08,3860.0,7.008263,7.008263,...,4812,Telcm,1971,0.753577,high,Telcm_high,0.000260,0.001902,0.004310,0.000048
4491,1971-01-29,NYSE,4832,STORER BROADCASTING CO,23347,23.0000,4.545500e-08,4223.0,2.600000,2.600000,...,4841,Telcm,1971,0.780264,high,Telcm_high,0.001139,0.008324,0.018861,0.000210
2867,1971-01-29,NYSE,4811,CONTINENTAL TELEPHONE CORP,20508,26.0000,2.970300e-08,26375.0,2.000000,2.000000,...,4813,Telcm,1971,0.719378,high,Telcm_high,0.008165,0.059666,0.135202,0.001503
4314,1971-01-29,AMEX,4832.0,SONDERLING BROADCASTING CORP,24790,28.1250,1.363640e-07,1006.0,1.500000,1.500000,...,4833,Telcm,1971,0.693171,high,Telcm_high,0.000305,0.002231,0.005055,0.000056
4760,1971-01-29,NYSE,4911.0,CENTRAL TEL & UTILS CORP,20410,21.6250,9.493700e-08,16588.0,3.375000,3.375000,...,4813,Telcm,1971,0.671265,high,Telcm_high,0.004017,0.029352,0.066511,0.000740
4192,1971-01-29,AMEX,3350,VIKOA INC,20022,9.2500,7.246400e-08,2316.0,0.445400,0.349387,...,4841,Telcm,1971,0.689590,high,Telcm_high,0.000192,0.001404,0.003181,0.000035
4605,1971-01-29,NYSE,4811.0,MID CONTINENT TEL CORP,20057,19.8750,1.438850e-07,6127.0,5.858716,4.770000,...,4812,Telcm,1971,0.729455,high,Telcm_high,0.001063,0.007765,0.017596,0.000196


In [ ]:
# Sort the data on w_sector
df = df.sort_values(['sector','Date'], ascending=True)

# Divide RET by 100
df['RET'] = df['RET'] / 100

display(df.head(1300))

,Date,EXCHCD,SICCD,COMNAM,LPERMCO,PRC,RET,SHROUT,CFACPR,CFACSHR,...,sic,sector,Year,DR,DR_port,DR_sector_port,w_DR,w_sector,w_DR_sector,w_overall
4509,1971-01-29,AMEX,3662.0,AIKEN INDUSTRIES INC,24779,4.000,1.851850e-07,2535.0,1.000000,1.000000,...,3812,BusEq,1971,0.424755,mid,BusEq_mid,0.000062,0.000154,0.000880,0.000019
4053,1971-01-29,NYSE,3499.0,DIEBOLD INC,20582,70.250,-3.103400e-08,3133.0,20.374365,20.374365,...,3578,BusEq,1971,0.352120,low,BusEq_mid,0.001015,0.004098,0.023371,0.000513
3730,1971-01-29,NYSE,3914.0,INSILCO CORP,22655,18.125,1.348840e-07,8262.0,1.710339,1.710339,...,3678,BusEq,1971,0.667091,high,BusEq_high,0.001633,0.002404,0.022983,0.000301
2691,1971-01-29,NYSE,3573.0,MOHAWK DATA SCIENCES CORP,21467,29.000,9.952600e-08,5484.0,1.000000,1.000000,...,3571,BusEq,1971,0.723516,high,BusEq_high,0.001773,0.002610,0.024952,0.000327
3819,1971-01-29,AMEX,7391.0,APPLIED DATA RESH INC,25025,7.750,5.500000e-07,966.0,2.315250,2.315250,...,7372,BusEq,1971,0.637770,high,BusEq_high,0.000059,0.000087,0.000833,0.000011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24985,1971-10-29,NYSE,3679,MALLORY P R & CO INC,23395,46.250,5.435000e-09,1971.0,2.000000,2.000000,...,3690,BusEq,1971,0.298264,low,BusEq_low,0.000372,0.001484,0.002175,0.000172
26472,1971-10-29,NYSE,3822.0,LEEDS & NORTHRUP CO,24655,30.125,-1.074070e-07,1988.0,1.650000,1.650000,...,3823,BusEq,1971,0.500717,mid,BusEq_mid,0.000360,0.001098,0.005417,0.000127
25419,1971-10-29,AMEX,3662.0,BARNES ENGINEERING CO,23632,11.500,-1.711710e-07,486.0,1.050000,1.050000,...,3812,BusEq,1971,0.335145,low,BusEq_low,0.000028,0.000110,0.000162,0.000013
24931,1971-10-29,NYSE,3714.0,ALLEN ELEC & EQUIP CO,20046,17.625,-1.132080e-07,2203.0,4.801318,4.345435,...,3663,BusEq,1971,0.547183,mid,BusEq_high,0.000212,0.000649,0.005653,0.000075


### Creation of portfolios
I calculate the weighted return, the weighted debt-ratios, the average debt-ratio and the number of companies within each portfolio. I subsequently save the monthly portfolio returns as a .csv file to be used in the analysis.

In [ ]:
# Define the aggregation function
def weighted_return_and_count(x):
    x = x.replace([np.inf, -np.inf], np.nan)
    weighted_return = (x['w_sector'] * x['RET']).sum()
    weighted_debt_ratio = (x['w_sector'] * x['DR']).sum()
    average_debt_ratio = x['DR'].mean()
    company_count = x['LPERMCO'].nunique()  # Count unique companies in each group
    return pd.Series([weighted_return, weighted_debt_ratio, average_debt_ratio, company_count], index=['weighted_return','weighted_debt_ratio','average_debt_ratio', 'company_count'])

# Apply custom function to calculate weighted monthly return and company count for each sector as a time series
sector_monthly_returns = df.groupby(['Date', 'sector']).apply(weighted_return_and_count).reset_index()

# Sorting df for better readability
sector_monthly_returns = sector_monthly_returns.sort_values(by=['sector', 'Date'])

# Printing the df
display(sector_monthly_returns.head())

# Save sector portfolios to a csv file
path_sector_monthly_returns = 'C:\\Users\\valde\\OneDrive\\01 Master Thesis\\01 Data\\Data 240402\\Monthly\\Sector Monthly Returns.csv'
sector_monthly_returns.to_csv(path_sector_monthly_returns, index=False)

,Date,sector,weighted_return,weighted_debt_ratio,average_debt_ratio,company_count
0,1971-01-29,BusEq,0.052212,0.360779,0.444632,118.0
12,1971-02-26,BusEq,0.028695,0.363002,0.444632,118.0
24,1971-03-31,BusEq,0.076852,0.361979,0.444632,118.0
36,1971-04-30,BusEq,0.020884,0.365469,0.452270,142.0
48,1971-05-28,BusEq,-0.063363,0.366232,0.452270,142.0


In [ ]:
# Sort the data on company count
sector_monthly_returns = sector_monthly_returns.sort_values(['company_count'], ascending=True)

# Printing the df
display(sector_monthly_returns.head(20))

,Date,sector,weighted_return,weighted_debt_ratio,average_debt_ratio,company_count
11,1971-01-29,Utils,-0.006760,0.675900,0.661161,8.0
35,1971-03-31,Utils,0.091244,0.675628,0.661161,8.0
23,1971-02-26,Utils,-0.003456,0.675263,0.661161,8.0
47,1971-04-30,Utils,0.017995,0.653312,0.644855,11.0
59,1971-05-28,Utils,-0.056976,0.653931,0.644855,11.0
71,1971-06-30,Utils,0.018173,0.654024,0.644855,11.0
83,1971-07-30,Utils,-0.040157,0.651988,0.644855,11.0
95,1971-08-31,Utils,0.008125,0.653208,0.644855,11.0
107,1971-09-30,Utils,-0.044691,0.650752,0.644855,11.0
119,1971-10-29,Utils,-0.057329,0.651618,0.644855,11.0


In [ ]:
# Define the aggregation function
def weighted_return_and_count(x):
    x = x.replace([np.inf, -np.inf], np.nan)
    weighted_return = (x['w_DR'] * x['RET']).sum()
    weighted_debt_ratio = (x['w_DR'] * x['DR']).sum()
    average_debt_ratio = x['DR'].mean()
    company_count = x['LPERMCO'].nunique()  # Count unique companies in each group
    return pd.Series([weighted_return, weighted_debt_ratio, average_debt_ratio, company_count], index=['weighted_return','weighted_debt_ratio','average_debt_ratio', 'company_count'])

# Apply the function to calculate weighted monthly return and company count for each portfolio as a time series
DR_monthly_returns = df.groupby(['Date', 'DR_port']).apply(weighted_return_and_count).reset_index()

# Sorting df for better readability
DR_monthly_returns = DR_monthly_returns.sort_values(by=['DR_port', 'Date'])

# Print the df
display(DR_monthly_returns.head())

# Save the DR portfolio to a csv file
path_DR_monthly_returns = 'C:\\Users\\valde\\OneDrive\\01 Master Thesis\\01 Data\\Data 240402\\Monthly\\DR Monthly Returns.csv'
DR_monthly_returns.to_csv(path_DR_monthly_returns, index=False)

,Date,DR_port,weighted_return,weighted_debt_ratio,average_debt_ratio,company_count
0,1971-01-29,low,0.030047,0.325783,0.280563,499.0
3,1971-02-26,low,0.025367,0.324304,0.280795,500.0
6,1971-03-31,low,0.051465,0.324634,0.280983,501.0
9,1971-04-30,low,0.035267,0.321522,0.279618,583.0
12,1971-05-28,low,-0.035476,0.321416,0.279911,585.0


In [ ]:
# Define the aggregation function
def weighted_return_and_count(x):
    x = x.replace([np.inf, -np.inf], np.nan)
    weighted_return = (x['w_DR_sector'] * x['RET']).sum()
    weighted_debt_ratio = (x['w_DR_sector'] * x['DR']).sum()
    average_debt_ratio = x['DR'].mean()
    company_count = x['LPERMCO'].nunique()  # Count unique companies in each group
    return pd.Series([weighted_return, weighted_debt_ratio, average_debt_ratio, company_count], index=['weighted_return','weighted_debt_ratio','average_debt_ratio', 'company_count'])

# Apply the function to calculate weighted monthly return and company count for each portfolio as a time series
DR_sector_monthly_returns = df.groupby(['Date', 'DR_sector_port']).apply(weighted_return_and_count).reset_index()

# Sorting the df for better readability (optional but recommended)
DR_sector_monthly_returns = DR_sector_monthly_returns.sort_values(by=['DR_sector_port', 'Date'])

# Print the df
display(DR_sector_monthly_returns.head())

# Save the DR portfolio to a csv file
path_DR_sector_monthly_returns = 'C:\\Users\\valde\\OneDrive\\01 Master Thesis\\01 Data\\Data 240402\\Monthly\\DR Sector Monthly Returns.csv'
DR_sector_monthly_returns.to_csv(path_DR_sector_monthly_returns, index=False)

,Date,DR_sector_port,weighted_return,weighted_debt_ratio,average_debt_ratio,company_count
0,1971-01-29,BusEq_high,0.073969,0.605304,0.664650,39.0
36,1971-02-26,BusEq_high,0.007356,0.606560,0.664650,39.0
72,1971-03-31,BusEq_high,0.121116,0.606250,0.664650,39.0
108,1971-04-30,BusEq_high,0.055236,0.612162,0.667118,47.0
144,1971-05-28,BusEq_high,-0.036784,0.609147,0.667118,47.0


In [ ]:
# Define the aggregation function
def weighted_return_and_count(x):
    x = x.replace([np.inf, -np.inf], np.nan)
    weighted_return = (x['w_overall'] * x['RET']).sum()
    weighted_debt_ratio = (x['w_overall'] * x['DR']).sum()
    average_debt_ratio = x['DR'].mean()
    company_count = x['LPERMCO'].nunique()
    return pd.Series([weighted_return, weighted_debt_ratio, average_debt_ratio, company_count],
                     index=['weighted_return', 'weighted_dr', 'avg_dr', 'company_count'])

# Apply the function to calculate weighted monthly return and company count for each portfolio as a time series
overall_df = df.groupby(['Date']).apply(weighted_return_and_count).reset_index()

# Sorting the df
overall_df = overall_df.sort_values(by=['Date'])

# Print the df
display(overall_df.head())

# Save the DR portfolio to a csv file
path_overall_df = 'C:\\Users\\valde\\OneDrive\\01 Master Thesis\\01 Data\\Data 240402\\Monthly\\Overall Monthly Returns.csv'
overall_df.to_csv(path_overall_df, index=False)

# Calculate the mean of the weighted_return
mean_weighted_return = overall_df['weighted_return'].mean()
print(mean_weighted_return)

,Date,weighted_return,weighted_dr,avg_dr,company_count
0,1971-01-29,0.045540,0.435730,0.479186,1492.0
1,1971-02-26,0.025018,0.437154,0.479600,1497.0
2,1971-03-31,0.049133,0.437678,0.480244,1504.0
3,1971-04-30,0.039937,0.437755,0.480035,1746.0
4,1971-05-28,-0.035038,0.437728,0.480335,1756.0


0.009418859065738375


In [ ]:
unique_count = df['LPERMCO'].nunique()

print("Number of unique values in 'LPERMCO':", unique_count)

# Group the data by the 'sector' and then apply a lambda function to count unique 'LPERMCO' values
unique_counts = df.groupby('sector')['LPERMCO'].nunique()

# Display the result
print(unique_counts)


Number of unique values in 'LPERMCO': 25026
sector
BusEq    4551
Chems     455
Durbl     599
Enrgy    1234
Hlth     3039
Manuf    2217
Money    4521
NoDur    1247
Other    3947
Shops    2352
Telcm     770
Utils     390
Name: LPERMCO, dtype: int64


In [ ]:
# Group the data by the 'sector' and then apply a lambda function to count unique 'LPERMCO' values
unique_counts = df.groupby('DR_sector_port')['LPERMCO'].nunique()

# store as data frame
unique_counts = pd.DataFrame(unique_counts)

# reset index
unique_counts = unique_counts.reset_index()

# Splitting the 'DR_sector_port' into 'sector' and 'suffix'
unique_counts['sector'] = unique_counts['DR_sector_port'].str.extract('(.*)_(high|mid|low)')[0]
unique_counts['suffix'] = unique_counts['DR_sector_port'].str.extract('(.*)_(high|mid|low)')[1]

# Defining a custom order for the suffixes
suffix_order = {'high': 0, 'mid': 1, 'low': 2}
unique_counts['suffix_rank'] = unique_counts['suffix'].map(suffix_order)

# Sorting the DataFrame by 'sector' and 'suffix_rank'
unique_counts = unique_counts.sort_values(by=['sector', 'suffix_rank'])

# Optionally, drop the helper columns if they are no longer needed
unique_counts = unique_counts.drop(['sector', 'suffix', 'suffix_rank'], axis=1)

# Display the result
display(unique_counts)

,DR_sector_port,LPERMCO
0,BusEq_high,3094
2,BusEq_mid,3529
1,BusEq_low,3221
3,Chems_high,318
5,Chems_mid,336
4,Chems_low,322
6,Durbl_high,436
8,Durbl_mid,452
7,Durbl_low,393
9,Enrgy_high,878
